In [1]:
import logging

import yaml
from nltk.tokenize import sent_tokenize
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
from slugify import slugify

## Logging

In [2]:
try:
    os.remove('new_ft_sentences.log')
except:
    pass

logging.getLogger().handlers = []
logging.getLogger('requests.packages.urllib3').setLevel(logging.WARNING)
logging.basicConfig(
    filename='new_ft_sentences.log',
    level=logging.INFO,
    format='%(asctime)s %(message)s'
)

## MongoDB

In [3]:
# for x in DB.rows.find({'pub_date': {'$gt': '20030103', '$lt': '20030107'}}):
#     print(x)

In [4]:
client = MongoClient()
client.drop_database('new_ft_sentences')
db_sentences = client.new_ft_sentences
db_text = client.new_ft_text

In [5]:
def insert_rows(db, rows):
    try:
        db.rows.insert_many(rows, ordered=False)
    except BulkWriteError as ex:
        for err in ex.details['writeErrors']:
            if err['code'] == 11000:
                _id = err['op']['_id']
                logging.info('BulkWriteError: {} - {}'.format(ex, _id))
                global dups
                dups += 1

## Search terms

In [6]:
with open('search_terms.yml') as search_term_file:
    term_yaml = yaml.load(search_term_file.read())

In [7]:
# [(search_term, original_term, term_type)]
search_terms_aux = [
    (t, k2, k1)
        for k1 in term_yaml
            for k2 in term_yaml[k1]
                for t in term_yaml[k1][k2]
]

In [8]:
# Remove
remove_list = ['Become Inc', 'Indeed', 'at&t', 'Signal']

In [9]:
search_terms = filter(lambda x: x[1] not in remove_list, search_terms_aux)
slugified_terms = list(map(lambda x: (slugify(x[0]), x[1], x[2]), search_terms))

## Main

In [10]:
%%time
for article in db_text.rows.find():
    rows = []
    text = article['text']
    sentences = sent_tokenize(text)
    for sentence in sentences:
        slugified_sentence = slugify(sentence)
        for slugified_term in slugified_terms:
            if slugified_term[0] in slugified_sentence:
                row = {
                    'article_id': article['_id'],
                    'date': article['date'],
                    'url': article['url'],
                    'source': article['source'],
                    'title': article['title'],
                    'term_category': slugified_term[2],
                    'term': slugified_term[1],
                    'sentence': sentence,
                }
                rows.append(row)
    if rows:
        insert_rows(db_sentences, rows)

CPU times: user 1h 17min 13s, sys: 13.7 s, total: 1h 17min 27s
Wall time: 1h 19min 11s


In [11]:
db_sentences.rows.count()

2000584

In [12]:
db_sentences.rows.find_one({'_id': ObjectId('56140caba688eb514e37573a')})

NameError: name 'ObjectId' is not defined